In [17]:
import tensorflow as tf
import numpy as np
from PIL import Image

from object_detection.utils import dataset_util

## Read datasets

In [4]:
import json

In [3]:
train_filename = "./data/train/train.json"
test_filename = "./data/test/test.json"

In [5]:
def read_json(filename):
    with open(filename) as json_data:
        data = json.load(json_data)
        return data

In [63]:
train_data = read_json(train_filename)
test_data = read_json(test_filename)

In [24]:
train_data[0]['image_path'].split('/')[-1]

'1.png'

In [25]:
train_data[0]

{'image_path': 'data/train/formated_data_192x64/1.png',
 'labels': [1.0, 9.0],
 'rects': [{'x1': 64.0, 'x2': 85.0, 'y1': 14.0, 'y2': 54.0},
  {'x1': 84.0, 'x2': 109.0, 'y1': 15.0, 'y2': 55.0}]}

In [30]:
type(b's')

bytes

In [32]:
type(bytes('filename.png', 'utf-8'))

bytes

In [49]:
bytes(bytes('filename.png', 'utf-8').decode('utf-8').replace('.png', ''), 'utf-8')

b'filename'

## Convert to TFRecord format

In [102]:
FINAL_HEIGHT = 64
FINAL_WIDTH = 192

def create_tf_example(example):
    height = FINAL_HEIGHT # Image height
    width = FINAL_WIDTH # Image width
    filename = bytes(example['image_path'], 'utf-8') # Filename of the image. Empty if image is not from file
    encoded_image_data = tf.gfile.FastGFile(example['image_path'], 'rb').read() # Encoded image bytes
    #Image.open(example['image_path'], 'r').tobytes()
    image_format = b'png' # b'jpeg' or b'png'

    xmins = [] # List of normalized left x coordinates in bounding box (1 per box)
    xmaxs = [] # List of normalized right x coordinates in bounding box
             # (1 per box)
    ymins = [] # List of normalized top y coordinates in bounding box (1 per box)
    ymaxs = [] # List of normalized bottom y coordinates in bounding box
             # (1 per box)
    classes_text = [] # List of string class name of bounding box (1 per box)
    classes = [] # List of integer class id of bounding box (1 per box)
    
    for rect, label in zip(example['rects'], example['labels']):
        label = int(label)
        xmins.append(rect['x1']/width)
        xmaxs.append(rect['x2']/width)
        ymins.append(rect['y1']/height)
        ymaxs.append(rect['y2']/height)
        if xmins[-1] < 0 or xmaxs[-1] > 1. or ymins[-1] < 0 or ymaxs[-1] > 1.:
            raise Exception()
        if xmaxs[-1] < 0 or xmins[-1] > 1. or ymaxs[-1] < 0 or ymins[-1] > 1.:
            raise Exception()
        classes_text.append(bytes(str(label), 'utf-8'))
        classes.append(label if label != 0 else 10)

    tf_example = tf.train.Example(features=tf.train.Features(feature={
        'image/height': dataset_util.int64_feature(height),
        'image/width': dataset_util.int64_feature(width),
        'image/filename': dataset_util.bytes_feature(filename),
        'image/source_id': dataset_util.bytes_feature(filename),
        'image/encoded': dataset_util.bytes_feature(encoded_image_data),
        'image/format': dataset_util.bytes_feature(image_format),
        'image/object/bbox/xmin': dataset_util.float_list_feature(xmins),
        'image/object/bbox/xmax': dataset_util.float_list_feature(xmaxs),
        'image/object/bbox/ymin': dataset_util.float_list_feature(ymins),
        'image/object/bbox/ymax': dataset_util.float_list_feature(ymaxs),
        'image/object/class/text': dataset_util.bytes_list_feature(classes_text),
        'image/object/class/label': dataset_util.int64_list_feature(classes),
    }))
    return tf_example

In [103]:
def write_TFRecord(examples, output_path):
    counter = 0
    with tf.python_io.TFRecordWriter(output_path) as writer:
        for example in examples:
            try:
                tf_example = create_tf_example(example)
                writer.write(tf_example.SerializeToString())
            except:
                counter += 1
    print(counter)

In [104]:
train_output_path = 'train.record'
test_output_path = 'test.record'

In [105]:
%%time
write_TFRecord(train_data, train_output_path)

180
CPU times: user 1min 27s, sys: 5.32 s, total: 1min 32s
Wall time: 1min 55s


In [106]:
%%time
write_TFRecord(test_data, test_output_path)

39
CPU times: user 29.9 s, sys: 1.93 s, total: 31.9 s
Wall time: 35.5 s


## Convert for images without resize

In [94]:
def create_tf_example(example):
    img = Image.open(example['image_path'], 'r')
    width, height = img.getbbox()[2:]
    filename = bytes(example['image_path'], 'utf-8') # Filename of the image. Empty if image is not from file
    encoded_image_data = tf.gfile.FastGFile(example['image_path'], 'rb').read() # Encoded image bytes
    #Image.open(example['image_path'], 'r').tobytes()
    image_format = b'png' # b'jpeg' or b'png'

    xmins = [] # List of normalized left x coordinates in bounding box (1 per box)
    xmaxs = [] # List of normalized right x coordinates in bounding box
             # (1 per box)
    ymins = [] # List of normalized top y coordinates in bounding box (1 per box)
    ymaxs = [] # List of normalized bottom y coordinates in bounding box
             # (1 per box)
    classes_text = [] # List of string class name of bounding box (1 per box)
    classes = [] # List of integer class id of bounding box (1 per box)
    
    for rect, label in zip(example['rects'], example['labels']):
        label = int(label)
        xmins.append(rect['x1']/width)
        xmaxs.append(rect['x2']/width)
        ymins.append(rect['y1']/height)
        ymaxs.append(rect['y2']/height)
        if xmins[-1] < 0 or xmaxs[-1] > 1. or ymins[-1] < 0 or ymaxs[-1] > 1.:
            raise Exception()
        if xmaxs[-1] < 0 or xmins[-1] > 1. or ymaxs[-1] < 0 or ymins[-1] > 1.:
            raise Exception()
        classes_text.append(bytes(str(label), 'utf-8'))
        classes.append(label if label != 0 else 10)

    tf_example = tf.train.Example(features=tf.train.Features(feature={
        'image/height': dataset_util.int64_feature(height),
        'image/width': dataset_util.int64_feature(width),
        'image/filename': dataset_util.bytes_feature(filename),
        'image/source_id': dataset_util.bytes_feature(filename),
        'image/encoded': dataset_util.bytes_feature(encoded_image_data),
        'image/format': dataset_util.bytes_feature(image_format),
        'image/object/bbox/xmin': dataset_util.float_list_feature(xmins),
        'image/object/bbox/xmax': dataset_util.float_list_feature(xmaxs),
        'image/object/bbox/ymin': dataset_util.float_list_feature(ymins),
        'image/object/bbox/ymax': dataset_util.float_list_feature(ymaxs),
        'image/object/class/text': dataset_util.bytes_list_feature(classes_text),
        'image/object/class/label': dataset_util.int64_list_feature(classes),
    }))
    return tf_example

In [97]:
train_without_resize_filename = "./data/train/train_without_resize.json"
test_without_resize_filename = "./data/test/test_without_resize.json"

In [98]:
train_without_resize_data = read_json(train_without_resize_filename)
test_without_resize_data = read_json(test_without_resize_filename)

In [99]:
train_without_resize_output_path = 'train_without_resize.record'
test_without_resize_output_path = 'test_without_resize.record'

In [100]:
%%time
write_TFRecord(train_without_resize_data, train_without_resize_output_path)

181
CPU times: user 1min 51s, sys: 7.5 s, total: 1min 58s
Wall time: 2min 53s


In [101]:
%%time
write_TFRecord(test_without_resize_data, test_without_resize_output_path)

39
CPU times: user 49.8 s, sys: 3.21 s, total: 53 s
Wall time: 1min 14s
